![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [694]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [695]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42

In [696]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [697]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')
cities = pd.read_excel('/kaggle/input/cities-info-2021/city.xlsx')

Посмотрим на датасет

In [698]:
df_train.info()

Видим пропущенные значения в трех колонках

In [699]:
df_test.info()

In [700]:
sample_submission.head(5)

In [701]:
sample_submission.info()

In [702]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [703]:
data.info()

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

Как видим, большинство признаков у нас требует очистки и предварительной обработки.

# Cleaning and Prepping Data
Обычно данные содержат в себе кучу мусора, который необходимо почистить, для того чтобы привести их в приемлемый формат. Чистка данных — это необходимый этап решения почти любой реальной задачи.   
![](https://analyticsindiamag.com/wp-content/uploads/2018/01/data-cleaning.png)

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [704]:
data.City.value_counts()

In [705]:
# Для примера я возьму столбец Number of Reviews
data['Number_of_Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [706]:
data['Number_of_Reviews_isNAN']

In [707]:
# Далее заполняем пропуски 0, вы можете попробовать заполнением средним или средним по городу и тд...
data['Number of Reviews'].fillna(0, inplace=True)

Так как заметно, что у некоторых ресторанов отличный рейтинг при 1-2 отзывах - добавим суммарное количество баллов рейтинга (чем больше отзывов - тем больше можно доверять рейтингу на сайте)

In [708]:
data['Rank1'] = data['Ranking'] * data['Number of Reviews']

In [709]:
data.sample(5)

Переименуем колонки с пробелами для удобства

In [710]:
data.rename(columns={'Cuisine Style': 'cuisine_style'}, inplace=True)
data.rename(columns={'Price Range': 'price_range'}, inplace=True)
data.rename(columns={'Number of Reviews': 'number_of_reviews'}, inplace=True)

Проверим, что ценовые категории не имеют выбросов

In [711]:
data.price_range.value_counts()

Проверим заполняемость цифровыми значениями категориального признака категории цены

In [712]:
data.price_range[7]

In [713]:
data['new_price'] = data['price_range'].apply(lambda x: 3 if x == '$$$$' else (1 if x == '$' else (2 if x == '$$ - $$$' else False)))

заменили категории цен на численные значения

In [714]:
data.head(3)

Проверим, что NA = False в новом столбце

In [715]:
data.new_price[7]

Заполним пропуски в категории цены по городам

In [716]:
data2 = data.copy(deep=True)
data2 = data2.dropna(axis = 0, subset = ['price_range'])

In [717]:
data2.new_price = pd.to_numeric(data2['new_price'])

In [718]:
data2.info()

In [719]:
s = data2.groupby('City').new_price.mean()
s

получили средние значения категории цены по городам, добавим в датафрейм

In [720]:
data['to_fill_NA_price'] = data.City.map(s, na_action=None)

In [721]:
def fillna_price(row):  
    if pd.isna(row.price_range) is False:
        return row.new_price
    else:
        return row.to_fill_NA_price
  
# Применяем функцию  
final_price = data.apply(lambda row: fillna_price(row), axis=1)  
  
# Заменяем старый столбец с пропусками на новый без пропусков  
data['final_price'] = final_price

Добавим численность населения и средний доход по городам

In [722]:
data = data.merge(cities, on = 'City', how = 'left')

Добавим признак столица / не столица

In [723]:
list_Of_NotCapitalCity = ['Barcelona', 'Milan', 'Hamburg', 'Munich', 
                          'Lyon', 'Zurich', 'Oporto', 'Geneva', 'Krakow']
data['Capital_City'] = data['City'].apply(lambda x: 0.0 if x in list_Of_NotCapitalCity else 1.0)
data

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [724]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

In [725]:
# для One-Hot Encoding применим get_dummies для городов
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

In [726]:
data.head(5)

# EDA 
[Exploratory Data Analysis](https://ru.wikipedia.org/wiki/Разведочный_анализ_данных) - Анализ данных
На этом этапе мы строим графики, ищем закономерности, аномалии, выбросы или связи между признаками.
В общем цель этого этапа понять, что эти данные могут нам дать и как признаки могут быть взаимосвязаны между собой.
Понимание изначальных признаков позволит сгенерировать новые, более сильные и, тем самым, сделать нашу модель лучше.
![](https://miro.medium.com/max/2598/1*RXdMb7Uk6mGqWqPguHULaQ.png)

### Посмотрим распределение признака

In [727]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [728]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [729]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [730]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

>Подумайте как из этого можно сделать признак для вашей модели. Я покажу вам пример, как визуализация помогает находить взаимосвязи. А далее действуйте без подсказок =) 


### Посмотрим распределение целевой переменной

In [731]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [732]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [733]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [734]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

Благодаря визуализации подтверждается гипотеза - что в Лондоне и Париже наибольшая численность населения, что численность населения влияет на ранг ресторанов, 

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [735]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
cities = pd.read_excel('/kaggle/input/cities-info-2021/city.xlsx')

df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [736]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    # убираем не нужные для модели признаки
    df_output.drop(['Restaurant_id','ID_TA',], axis = 1, inplace=True)
    
    
    # ################### 2. NAN ############################################################## 
    # Далее заполняем пропуски, вы можете попробовать заполнением средним или средним по городу и тд...
    df_output['Number of Reviews'].fillna(0, inplace=True)
    df_output.rename(columns={'Cuisine Style': 'cuisine_style'}, inplace=True)
    df_output.rename(columns={'Price Range': 'price_range'}, inplace=True)
    df_output.rename(columns={'Number of Reviews': 'number_of_reviews'}, inplace=True)
    #заполянем пропуски в ценовой категории средним значением по городу
    df_output['new_price'] = df_output['price_range'].apply(lambda x: 3 if x == '$$$$' else (1 if x == '$' else (2 if x == '$$ - $$$' else False)))
    data2 = df_output.copy(deep=True)
    data2 = data2.dropna(axis = 0, subset = ['price_range'])
    data2.new_price = pd.to_numeric(data2['new_price'])
    s = data2.groupby('City').new_price.mean()
    df_output['to_fill_NA_price'] = df_output.City.map(s, na_action=None)
    def fillna_price(row):  
        if pd.isna(row.price_range) is False:
            return row.new_price
        else:
            return row.to_fill_NA_price
  
    # Применяем функцию  
    final_price = df_output.apply(lambda row: fillna_price(row), axis=1)  

    # Заменяем старый столбец с пропусками на новый без пропусков  
    df_output['final_price'] = final_price
    df_output = df_output.drop(columns = ['to_fill_NA_price', 'new_price'])
    
    
    # ################### 3. Encoding ############################################################## 
    # для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
    df_output = df_output.merge(cities, on = 'City', how = 'left')
    list_Of_NotCapitalCity = ['Barcelona', 'Milan', 'Hamburg', 'Munich', 
                          'Lyon', 'Zurich', 'Oporto', 'Geneva', 'Krakow']
    df_output['Capital_City'] = df_output['City'].apply(lambda x: 0.0 if x in list_Of_NotCapitalCity else 1.0)
    Rank_average = df_output.groupby(['City'])['Ranking'].mean()
    df_output['Rank_average'] = df_output['City'].apply(lambda x: Rank_average[x])

    #применяем get_dummies для городов
    df_output = pd.get_dummies(df_output, columns=[ 'City',], dummy_na=True)
    
    
    # ################### 4. Feature Engineering ####################################################
    #добавим ссумарное значение рейтингов (чем больше оценок, тем больше доверия к рейтингу)
    df_output['Rank1'] = df_output['Ranking'] * df_output['number_of_reviews']
    #учтем, что в зависимости от численности населения суммарное количество отзывов, как и суммарный рейтинг будут разными
    df_output['Rank2'] = df_output['Rank1'] / df_output['Population']
    #приведем ранг ресторанов к среднему рангу по городу
    df_output['Rank3'] = df_output['Ranking'] / df_output['Rank_average']
    
    
    # ################### 5. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [737]:
df_preproc = preproc_data(data)
df_preproc.sample(11)

In [738]:
df_preproc.info()

In [739]:
# Теперь выделим тестовую часть
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [740]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [741]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [742]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [743]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [744]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [745]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [746]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [747]:
test_data.sample(10)

In [748]:
test_data = test_data.drop(['Rating'], axis=1)

In [749]:
sample_submission

In [750]:
predict_submission = model.predict(test_data)

In [751]:
predict_submission

In [752]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
